Code for Code Explaination


In [29]:
import requests
import json
import os
from google.colab import userdata
from google.colab import auth
import gspread
from google.auth import default

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

def get_openai_response(prompt_text: str,  api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str, api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})

    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

def add_to_google_sheet(data):
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Create a new spreadsheet or open an existing one
    try:
        sh = gc.open("LLM_Code_Analysis")
    except gspread.exceptions.SpreadsheetNotFound:
        sh = gc.create("LLM_Code_Analysis")
        # Share with yourself to have permission to view it in Google Drive
        sh.share(userdata.get('EMAIL'), perm_type='user', role='writer')

    worksheet = sh.sheet1

    # Append data to the next available row
    worksheet.append_row(data)


# --- Main Logic ---

def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    user_prompt = """
    In a GitHub repo titled Keel the following code files exist 'src/main/java/com/harium/keel/effect/height/Sobel.java'

    Explain the file called: sobel.java.First in 40-50 words explain what this code file does in the context of the project repository. Then, explain what this code file does(write it in 100-120 words and write 1 small line describing each method in the code.)
    package com.harium.keel.effect.height; First in 40-50 words explain what this code file does in the context of the project repository. Then, explain what this code file does(write it in 100-120 words and write 1 small line describing each method in the code.)
    import com.harium.keel.core.Effect;
    import com.harium.keel.core.source.ImageSource;
    import com.harium.keel.core.source.MatrixSource;

    /**
     * Sobel Filter
     * Code from: https://stackoverflow.com/a/30511674
     */
    public class Sobel implements Effect {

        /**
         * Expects a height mat as input
         *
         * @param input - A grayscale height map
         * @return edges
         */
        @Override
        public ImageSource apply(ImageSource input) {
            final int[][] pixelMatrix = new int[3][3];

            int w = input.getWidth();
            int h = input.getHeight();

            int[][] output = new int[h][w];

            for (int j = 1; j < h - 1; j++) {
                for (int i = 1; i < w - 1; i++) {
                    pixelMatrix[0][0] = input.getR(i - 1, j - 1);
                    pixelMatrix[0][1] = input.getRGB(i - 1, j);
                    pixelMatrix[0][2] = input.getRGB(i - 1, j + 1);
                    pixelMatrix[1][0] = input.getRGB(i, j - 1);
                    pixelMatrix[1][2] = input.getRGB(i, j + 1);
                    pixelMatrix[2][0] = input.getRGB(i + 1, j - 1);
                    pixelMatrix[2][1] = input.getRGB(i + 1, j);
                    pixelMatrix[2][2] = input.getRGB(i + 1, j + 1);

                    int edge = (int) convolution(pixelMatrix);
                    int rgb = (edge << 16 | edge << 8 | edge);
                    output[j][i] = rgb;
                }
            }

            MatrixSource source = new MatrixSource(output);
            return source;
        }

        private static double convolution(int[][] pixelMatrix) {
            int gy = (pixelMatrix[0][0] * -1) + (pixelMatrix[0][1] * -2) + (pixelMatrix[0][2] * -1) + (pixelMatrix[2][0]) + (pixelMatrix[2][1] * 2) + (pixelMatrix[2][2] * 1);
            int gx = (pixelMatrix[0][0]) + (pixelMatrix[0][2] * -1) + (pixelMatrix[1][0] * 2) + (pixelMatrix[1][2] * -2) + (pixelMatrix[2][0]) + (pixelMatrix[2][2] * -1);

            return Math.sqrt(Math.pow(gy, 2) + Math.pow(gx, 2));
        }

    }in context of the project.
    """



    print("\nGetting responses from LLMs...\n")

    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, OPENAI_API_KEY)
    print(gpt_response)
    print("-" * 30)
    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt,  DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)

    # Add results to Google Sheet
    sheet_data = ["Code Explanation", gpt_response, deepseek_response, gemini_response]
    add_to_google_sheet(sheet_data)
    print("Results added to Google Sheet.")


if __name__ == "__main__":
    main()

Welcome to the LLM Code Analyzer!
------------------------------

Getting responses from LLMs...

--- GPT-4o Response ---
### Context of the Project Repository
In the context of the Keel project repository, `Sobel.java` implements an edge detection algorithm using the Sobel filter. This code processes height maps in greyscale to enhance and outline features within the image, contributing to any image processing tasks that require edge detection or analysis.

### Explanation of the Code
The `Sobel.java` file is part of the `com.harium.keel.effect.height` package and implements edge detection using the Sobel filter. As an implementation of the `Effect` interface, it takes a grayscale height map as input (`ImageSource`) and returns an image highlighting edges (`MatrixSource`). The algorithm iterates over pixel values, applying the Sobel operator to compute the gradient magnitudes in both x and y directions. This involves a convolution approach with pre-defined Sobel matrices to detect int

Code for Code Quality Metrics



In [25]:
import requests
import json
import os
from google.colab import userdata
from google.colab import auth
import gspread
from google.auth import default


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str,  api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})


    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

def add_to_google_sheet(data):
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Create a new spreadsheet or open an existing one
    try:
        sh = gc.open("LLM_Code_Analysis")
    except gspread.exceptions.SpreadsheetNotFound:
        sh = gc.create("LLM_Code_Analysis")
        # Share with yourself to have permission to view it in Google Drive
        sh.share(userdata.get('EMAIL'), perm_type='user', role='writer')

    worksheet = sh.sheet1

    # Append data to the next available row
    worksheet.append_row(data)


# --- Main Logic ---


def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    user_prompt = """
"CVE categories :Code Execution,Denial of Service,Information Leak,Privilege Escalation,Overflow,ByPass, Memory Corruption. Calculate the cyclomatic complexity, maintainability index and vulnerability category that suit most from CVE categories for the following code. Just check out the codes and from the code try to answer . just write the numbers:\n""#include <linux/errno.h>
#include <linux/fs.h>
#include <linux/slab.h>
#include <linux/random.h>

#include """"hfsplus_fs.h""""
#include """"hfsplus_raw.h""""

static inline void hfsplus_instantiate(struct dentry *dentry,
                                       struct inode *inode, u32 cnid)
{
        dentry->d_fsdata = (void *)(unsigned long)cnid;
        d_instantiate(dentry, inode);
}

/* Find the entry inside dir named dentry->d_name */
static struct dentry *hfsplus_lookup(struct inode *dir, struct dentry *dentry,
                                     struct nameidata *nd)
{
        struct inode *inode = NULL;
        struct hfs_find_data fd;
        struct super_block *sb;
        hfsplus_cat_entry entry;
        int err;
        u32 cnid, linkid = 0;
        u16 type;

        sb = dir->i_sb;

        dentry->d_fsdata = NULL;
        err = hfs_find_init(HFSPLUS_SB(sb)->cat_tree, &fd);
        if (err)
                return ERR_PTR(err);
        hfsplus_cat_build_key(sb, fd.search_key, dir->i_ino, &dentry->d_name);
again:
        err = hfs_brec_read(&fd, &entry, sizeof(entry));
        if (err) {
                if (err == -ENOENT) {
                        hfs_find_exit(&fd);
                        /* No such entry */
                        inode = NULL;
                        goto out;
                }
                goto fail;
        }
        type = be16_to_cpu(entry.type);
        if (type == HFSPLUS_FOLDER) {
                if (fd.entrylength < sizeof(struct hfsplus_cat_folder)) {
                        err = -EIO;
                        goto fail;
                }
                cnid = be32_to_cpu(entry.folder.id);
                dentry->d_fsdata = (void *)(unsigned long)cnid;
        } else if (type == HFSPLUS_FILE) {
                if (fd.entrylength < sizeof(struct hfsplus_cat_file)) {
                        err = -EIO;
                        goto fail;
                }
                cnid = be32_to_cpu(entry.file.id);
                if (entry.file.user_info.fdType ==
                                cpu_to_be32(HFSP_HARDLINK_TYPE) &&
                                entry.file.user_info.fdCreator ==
                                cpu_to_be32(HFSP_HFSPLUS_CREATOR) &&
                                (entry.file.create_date ==
                                        HFSPLUS_I(HFSPLUS_SB(sb)->hidden_dir)->
                                                create_date ||
                                entry.file.create_date ==
                                        HFSPLUS_I(sb->s_root->d_inode)->
                                                create_date) &&
                                HFSPLUS_SB(sb)->hidden_dir) {
                        struct qstr str;
                        char name[32];

                        if (dentry->d_fsdata) {
                                /*
                                 * We found a link pointing to another link,
                                 * so ignore it and treat it as regular file.
                                 */
                                cnid = (unsigned long)dentry->d_fsdata;
                                linkid = 0;
                        } else {
                                dentry->d_fsdata = (void *)(unsigned long)cnid;
                                linkid =
                                        be32_to_cpu(entry.file.permissions.dev);
                                str.len = sprintf(name, """"iNode%d"""", linkid);
                                str.name = name;
                                hfsplus_cat_build_key(sb, fd.search_key,
                                        HFSPLUS_SB(sb)->hidden_dir->i_ino,
                                        &str);
                                goto again;
                        }
                } else if (!dentry->d_fsdata)
                        dentry->d_fsdata = (void *)(unsigned long)cnid;
        } else {
                printk(KERN_ERR """"hfs: invalid catalog entry type in lookup\n"""");
                err = -EIO;
                goto fail;
        }
        hfs_find_exit(&fd);
        inode = hfsplus_iget(dir->i_sb, cnid);
        if (IS_ERR(inode))
                return ERR_CAST(inode);
        if (S_ISREG(inode->i_mode))
                HFSPLUS_I(inode)->linkid = linkid;
out:
        d_add(dentry, inode);
        return NULL;
fail:
        hfs_find_exit(&fd);
        return ERR_PTR(err);
}

static int hfsplus_readdir(struct file *filp, void *dirent, filldir_t filldir)
{
        struct inode *inode = filp->f_path.dentry->d_inode;
        struct super_block *sb = inode->i_sb;
        int len, err;
        char strbuf[HFSPLUS_MAX_STRLEN + 1];
        hfsplus_cat_entry entry;
        struct hfs_find_data fd;
        struct hfsplus_readdir_data *rd;
        u16 type;

        if (filp->f_pos >= inode->i_size)
                return 0;

        err = hfs_find_init(HFSPLUS_SB(sb)->cat_tree, &fd);
        if (err)
                return err;
        hfsplus_cat_build_key(sb, fd.search_key, inode->i_ino, NULL);
        err = hfs_brec_find(&fd);
        if (err)
                goto out;

        switch ((u32)filp->f_pos) {
        case 0:
                /* This is completely artificial... */
                if (filldir(dirent, """"."""", 1, 0, inode->i_ino, DT_DIR))
                        goto out;
                filp->f_pos++;
                /* fall through */
        case 1:
                if (fd.entrylength > sizeof(entry) || fd.entrylength < 0) {
                        err = -EIO;
                        goto out;
                }

                hfs_bnode_read(fd.bnode, &entry, fd.entryoffset,
                        fd.entrylength);
                if (be16_to_cpu(entry.type) != HFSPLUS_FOLDER_THREAD) {
                        printk(KERN_ERR """"hfs: bad catalog folder thread\n"""");
                        err = -EIO;
                        goto out;
                }
                if (fd.entrylength < HFSPLUS_MIN_THREAD_SZ) {
                        printk(KERN_ERR """"hfs: truncated catalog thread\n"""");
                        err = -EIO;
                        goto out;
                }
                if (filldir(dirent, """".."""", 2, 1,
                            be32_to_cpu(entry.thread.parentID), DT_DIR))
                        goto out;
                filp->f_pos++;
                /* fall through */
        default:
                if (filp->f_pos >= inode->i_size)
                        goto out;
                err = hfs_brec_goto(&fd, filp->f_pos - 1);
                if (err)
                        goto out;
        }

        for (;;) {
                if (be32_to_cpu(fd.key->cat.parent) != inode->i_ino) {
                        printk(KERN_ERR """"hfs: walked past end of dir\n"""");
                        err = -EIO;
                        goto out;
                }

                if (fd.entrylength > sizeof(entry) || fd.entrylength < 0) {
                        err = -EIO;
                        goto out;
                }

                hfs_bnode_read(fd.bnode, &entry, fd.entryoffset,
                        fd.entrylength);
                type = be16_to_cpu(entry.type);
                len = HFSPLUS_MAX_STRLEN;
                err = hfsplus_uni2asc(sb, &fd.key->cat.name, strbuf, &len);
                if (err)
                        goto out;
                if (type == HFSPLUS_FOLDER) {
                        if (fd.entrylength <
                                        sizeof(struct hfsplus_cat_folder)) {
                                printk(KERN_ERR """"hfs: small dir entry\n"""");
                                err = -EIO;
                                goto out;
                        }
                        if (HFSPLUS_SB(sb)->hidden_dir &&
                            HFSPLUS_SB(sb)->hidden_dir->i_ino ==
                                        be32_to_cpu(entry.folder.id))
                                goto next;
                        if (filldir(dirent, strbuf, len, filp->f_pos,
                                    be32_to_cpu(entry.folder.id), DT_DIR))
                                break;
                } else if (type == HFSPLUS_FILE) {
                        if (fd.entrylength < sizeof(struct hfsplus_cat_file)) {
                                printk(KERN_ERR """"hfs: small file entry\n"""");
                                err = -EIO;
                                goto out;
                        }
                        if (filldir(dirent, strbuf, len, filp->f_pos,
                                    be32_to_cpu(entry.file.id), DT_REG))
                                break;
                } else {
                        printk(KERN_ERR """"hfs: bad catalog entry type\n"""");
                        err = -EIO;
                        goto out;
                }
next:
                filp->f_pos++;
                if (filp->f_pos >= inode->i_size)
                        goto out;
                err = hfs_brec_goto(&fd, 1);
                if (err)
                        goto out;
        }
        rd = filp->private_data;
        if (!rd) {
                rd = kmalloc(sizeof(struct hfsplus_readdir_data), GFP_KERNEL);
                if (!rd) {
                        err = -ENOMEM;
                        goto out;
                }
                filp->private_data = rd;
                rd->file = filp;
                list_add(&rd->list, &HFSPLUS_I(inode)->open_dir_list);
        }
        memcpy(&rd->key, fd.key, sizeof(struct hfsplus_cat_key));
out:
        hfs_find_exit(&fd);
        return err;
}

static int hfsplus_dir_release(struct inode *inode, struct file *file)
{
        struct hfsplus_readdir_data *rd = file->private_data;
        if (rd) {
                mutex_lock(&inode->i_mutex);
                list_del(&rd->list);
                mutex_unlock(&inode->i_mutex);
                kfree(rd);
        }
        return 0;
}

static int hfsplus_link(struct dentry *src_dentry, struct inode *dst_dir,
                        struct dentry *dst_dentry)
{
        struct hfsplus_sb_info *sbi = HFSPLUS_SB(dst_dir->i_sb);
        struct inode *inode = src_dentry->d_inode;
        struct inode *src_dir = src_dentry->d_parent->d_inode;
        struct qstr str;
        char name[32];
        u32 cnid, id;
        int res;

        if (HFSPLUS_IS_RSRC(inode))
                return -EPERM;
        if (!S_ISREG(inode->i_mode))
                return -EPERM;

        mutex_lock(&sbi->vh_mutex);
        if (inode->i_ino == (u32)(unsigned long)src_dentry->d_fsdata) {
                for (;;) {
                        get_random_bytes(&id, sizeof(cnid));
                        id &= 0x3fffffff;
                        str.name = name;
                        str.len = sprintf(name, """"iNode%d"""", id);
                        res = hfsplus_rename_cat(inode->i_ino,
                                                 src_dir, &src_dentry->d_name,
                                                 sbi->hidden_dir, &str);
                        if (!res)
                                break;
                        if (res != -EEXIST)
                                goto out;
                }
                HFSPLUS_I(inode)->linkid = id;
                cnid = sbi->next_cnid++;
                src_dentry->d_fsdata = (void *)(unsigned long)cnid;
                res = hfsplus_create_cat(cnid, src_dir,
                        &src_dentry->d_name, inode);
                if (res)
                        /* panic? */
                        goto out;
                sbi->file_count++;
        }
        cnid = sbi->next_cnid++;
        res = hfsplus_create_cat(cnid, dst_dir, &dst_dentry->d_name, inode);
        if (res)
                goto out;

        inc_nlink(inode);
        hfsplus_instantiate(dst_dentry, inode, cnid);
        ihold(inode);
        inode->i_ctime = CURRENT_TIME_SEC;
        mark_inode_dirty(inode);
        sbi->file_count++;
        dst_dir->i_sb->s_dirt = 1;
out:
        mutex_unlock(&sbi->vh_mutex);
        return res;
}

static int hfsplus_unlink(struct inode *dir, struct dentry *dentry)
{
        struct hfsplus_sb_info *sbi = HFSPLUS_SB(dir->i_sb);
        struct inode *inode = dentry->d_inode;
        struct qstr str;
        char name[32];
        u32 cnid;
        int res;

        if (HFSPLUS_IS_RSRC(inode))
                return -EPERM;

        mutex_lock(&sbi->vh_mutex);
        cnid = (u32)(unsigned long)dentry->d_fsdata;
        if (inode->i_ino == cnid &&
            atomic_read(&HFSPLUS_I(inode)->opencnt)) {
                str.name = name;
                str.len = sprintf(name, """"temp%lu"""", inode->i_ino);
                res = hfsplus_rename_cat(inode->i_ino,
                                         dir, &dentry->d_name,
                                         sbi->hidden_dir, &str);
                if (!res) {
                        inode->i_flags |= S_DEAD;
                        drop_nlink(inode);
                }
                goto out;
        }
        res = hfsplus_delete_cat(cnid, dir, &dentry->d_name);
        if (res)
                goto out;

        if (inode->i_nlink > 0)
                drop_nlink(inode);
        if (inode->i_ino == cnid)
                clear_nlink(inode);
        if (!inode->i_nlink) {
                if (inode->i_ino != cnid) {
                        sbi->file_count--;
                        if (!atomic_read(&HFSPLUS_I(inode)->opencnt)) {
                                res = hfsplus_delete_cat(inode->i_ino,
                                                         sbi->hidden_dir,
                                                         NULL);
                                if (!res)
                                        hfsplus_delete_inode(inode);
                        } else
                                inode->i_flags |= S_DEAD;
                } else
                        hfsplus_delete_inode(inode);
        } else
                sbi->file_count--;
        inode->i_ctime = CURRENT_TIME_SEC;
        mark_inode_dirty(inode);
out:
        mutex_unlock(&sbi->vh_mutex);
        return res;
}

static int hfsplus_rmdir(struct inode *dir, struct dentry *dentry)
{
        struct hfsplus_sb_info *sbi = HFSPLUS_SB(dir->i_sb);
        struct inode *inode = dentry->d_inode;
        int res;

        if (inode->i_size != 2)
                return -ENOTEMPTY;

        mutex_lock(&sbi->vh_mutex);
        res = hfsplus_delete_cat(inode->i_ino, dir, &dentry->d_name);
        if (res)
                goto out;
        clear_nlink(inode);
        inode->i_ctime = CURRENT_TIME_SEC;
        hfsplus_delete_inode(inode);
        mark_inode_dirty(inode);
out:
        mutex_unlock(&sbi->vh_mutex);
        return res;
}

static int hfsplus_symlink(struct inode *dir, struct dentry *dentry,
                           const char *symname)
{
        struct hfsplus_sb_info *sbi = HFSPLUS_SB(dir->i_sb);
        struct inode *inode;
        int res = -ENOSPC;

        mutex_lock(&sbi->vh_mutex);
        inode = hfsplus_new_inode(dir->i_sb, S_IFLNK | S_IRWXUGO);
        if (!inode)
                goto out;

        res = page_symlink(inode, symname, strlen(symname) + 1);
        if (res)
                goto out_err;

        res = hfsplus_create_cat(inode->i_ino, dir, &dentry->d_name, inode);
        if (res)
                goto out_err;

        hfsplus_instantiate(dentry, inode, inode->i_ino);
        mark_inode_dirty(inode);
        goto out;

out_err:
        clear_nlink(inode);
        hfsplus_delete_inode(inode);
        iput(inode);
out:
        mutex_unlock(&sbi->vh_mutex);
        return res;
}

static int hfsplus_mknod(struct inode *dir, struct dentry *dentry,
                         umode_t mode, dev_t rdev)
{
        struct hfsplus_sb_info *sbi = HFSPLUS_SB(dir->i_sb);
        struct inode *inode;
        int res = -ENOSPC;

        mutex_lock(&sbi->vh_mutex);
        inode = hfsplus_new_inode(dir->i_sb, mode);
        if (!inode)
                goto out;

        if (S_ISBLK(mode) || S_ISCHR(mode) || S_ISFIFO(mode) || S_ISSOCK(mode))
                init_special_inode(inode, mode, rdev);

        res = hfsplus_create_cat(inode->i_ino, dir, &dentry->d_name, inode);
        if (res) {
                clear_nlink(inode);
                hfsplus_delete_inode(inode);
                iput(inode);
                goto out;
        }

        hfsplus_instantiate(dentry, inode, inode->i_ino);
        mark_inode_dirty(inode);
out:
        mutex_unlock(&sbi->vh_mutex);
        return res;
}

static int hfsplus_create(struct inode *dir, struct dentry *dentry, umode_t mode,
                          struct nameidata *nd)
{
        return hfsplus_mknod(dir, dentry, mode, 0);
}

static int hfsplus_mkdir(struct inode *dir, struct dentry *dentry, umode_t mode)
{
        return hfsplus_mknod(dir, dentry, mode | S_IFDIR, 0);
}

static int hfsplus_rename(struct inode *old_dir, struct dentry *old_dentry,
                          struct inode *new_dir, struct dentry *new_dentry)
{
        int res;

        /* Unlink destination if it already exists */
        if (new_dentry->d_inode) {
                if (S_ISDIR(new_dentry->d_inode->i_mode))
                        res = hfsplus_rmdir(new_dir, new_dentry);
                else
                        res = hfsplus_unlink(new_dir, new_dentry);
                if (res)
                        return res;
        }

        res = hfsplus_rename_cat((u32)(unsigned long)old_dentry->d_fsdata,
                                 old_dir, &old_dentry->d_name,
                                 new_dir, &new_dentry->d_name);
        if (!res)
                new_dentry->d_fsdata = old_dentry->d_fsdata;
        return res;
}

const struct inode_operations hfsplus_dir_inode_operations = {
        .lookup                = hfsplus_lookup,
        .create                = hfsplus_create,
        .link                = hfsplus_link,
        .unlink                = hfsplus_unlink,
        .mkdir                = hfsplus_mkdir,
        .rmdir                = hfsplus_rmdir,
        .symlink        = hfsplus_symlink,
        .mknod                = hfsplus_mknod,
        .rename                = hfsplus_rename,
};

const struct file_operations hfsplus_dir_operations = {
        .fsync                = hfsplus_file_fsync,
        .read                = generic_read_dir,
        .readdir        = hfsplus_readdir,
        .unlocked_ioctl = hfsplus_ioctl,
        .llseek                = generic_file_llseek,
        .release        = hfsplus_dir_release,
};""\n\nIn your response only give the detected values of these attributes. Don't give any explanation"
"""







    print("\nGetting responses from LLMs...\n")

    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, OPENAI_API_KEY)
    print(gpt_response)
    print("-" * 30)

    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt, DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)

     # Add results to Google Sheet
    sheet_data = ["Code Explanation", gpt_response, deepseek_response, gemini_response]
    add_to_google_sheet(sheet_data)
    print("Results added to Google Sheet.")

if __name__ == "__main__":
    main()

Welcome to the LLM Code Analyzer!
------------------------------

Getting responses from LLMs...

--- GPT-4o Response ---
Cyclomatic Complexity: 22  
Maintainability Index: 90  
Vulnerability Category: Memory Corruption
------------------------------


Code for Code Refactoring


In [ ]:
import requests
import json
import os
from google.colab import userdata
from google.colab import auth
import gspread
from google.auth import default


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# API Endpoints
OPENAI_ENDPOINT = "https://api.openai.com/v1/chat/completions"
DEEPSEEK_ENDPOINT = "https://api.deepseek.com/chat/completions"
GEMINI_ENDPOINT = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"

# --- Helper Functions for LLM Interactions ---

def get_openai_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "OPENAI_API_KEY":
        return "OpenAI API Key not configured. Please set OPENAI_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(OPENAI_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenAI API: {e}"
    except KeyError:
        return f"Unexpected response format from OpenAI API: {response.text}"

def get_deepseek_response(prompt_text: str, api_key: str) -> str:

    if not api_key or api_key == "YOUR_DEEPSEEK_API_KEY":
        return "Deepseek API Key not configured. Please set DEEPSEEK_API_KEY."

    messages = [
        {"role": "user", "content": prompt_text},

    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "deepseek-chat",
        "messages": messages,
        "max_tokens": 1000
    }

    try:
        response = requests.post(DEEPSEEK_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Deepseek API: {e}"
    except KeyError:
        return f"Unexpected response format from Deepseek API: {response.text}"

def get_gemini_response(prompt_text: str,  api_key: str) -> str:

    chat_history = []
    chat_history.append({"role": "user", "parts": [{"text": prompt_text}]})


    payload = {"contents": chat_history}
    api_url = f"{GEMINI_ENDPOINT}?key={api_key}"

    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        result = response.json()

        if result.get('candidates') and result['candidates'][0].get('content') and \
           result['candidates'][0]['content'].get('parts') and \
           result['candidates'][0]['content']['parts'][0].get('text'):
            return result['candidates'][0]['content']['parts'][0]['text'].strip()
        else:
            return f"Unexpected response format from Gemini API: {response.text}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to Gemini API: {e}"
    except Exception as e:
        return f"An unexpected error occurred with Gemini API: {e}"

def add_to_google_sheet(data):
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Create a new spreadsheet or open an existing one
    try:
        sh = gc.open("LLM_Code_Analysis")
    except gspread.exceptions.SpreadsheetNotFound:
        sh = gc.create("LLM_Code_Analysis")
        # Share with yourself to have permission to view it in Google Drive
        sh.share(userdata.get('EMAIL'), perm_type='user', role='writer')

    worksheet = sh.sheet1

    # Append data to the next available row
    worksheet.append_row(data)

# --- Main Logic ---


def main():
    print("Welcome to the LLM Code Analyzer!")
    print("-" * 30)

    user_prompt = """
Refactor the following code file and provide the complete, cleaned, and refactored version as output.
Include brief comments (12 to 15 words) next to each section where refactoring was performed,
explaining the changes made. Do not include any additional explanations or descriptions,
just give the refactored code as output:

public Object streamLoad(HttpServletRequest request,
                         @PathVariable(value = DB_KEY) String db,
                         @PathVariable(value = TABLE_KEY) String table) {
    boolean groupCommit = false;
    String groupCommitStr = request.getHeader("group_commit");
    if (groupCommitStr != null && groupCommitStr.equals("async_mode")) {
        groupCommit = true;
        try {
            String[] pair = new String[] {db, table};
            if (isGroupCommitBlock(pair)) {
                String msg = "insert table " + pair[1] + " is blocked on schema change";
                return new RestBaseResult(msg);
            }
        } catch (Exception e) {
            // handle
        }
    }
}

@RequestMapping(path = "/api/_http_stream", method = RequestMethod.PUT)
public Object streamLoadWithSql(HttpServletRequest request, HttpServletResponse response) {
    String sql = request.getHeader("sql");
    LOG.info("streaming load sql={}", sql);
    boolean groupCommit = false;
    String groupCommitStr = request.getHeader("group_commit");
    if (groupCommitStr != null && groupCommitStr.equals("async_mode")) {
        groupCommit = true;
        try {
            String[] pair = parseDbAndTb(sql);
            if (isGroupCommitBlock(pair)) {
                String msg = "insert table " + pair[1] + " is blocked on schema change";
                return new RestBaseResult(msg);
            }
        } catch (Exception e) {
            // handle
        }
    }
}

private boolean isGroupCommitBlock(String[] pair) throws TException {
    String fullDbName = getFullDbName(pair[0]);
    Database dbObj = Env.getCurrentInternalCatalog()
                        .getDbOrException(fullDbName, s -> new TException("database is invalid for dbName: " + s));
    Table tblObj = dbObj.getTableOrException(pair[1], s -> new TException("table is invalid: " + s));
    return Env.getCurrentEnv().getGroupCommitManager().isBlock(tblObj.getId());
}
"""







    print("\nGetting responses from LLMs...\n")

    # Get response from GPT-4o
    print("--- GPT-4o Response ---")
    gpt_response = get_openai_response(user_prompt, OPENAI_API_KEY)
    print(gpt_response)
    print("-" * 30)

    # Get response from Deepseek
    print("\n--- Deepseek Response ---")
    deepseek_response = get_deepseek_response(user_prompt, DEEPSEEK_API_KEY)
    print(deepseek_response)
    print("-" * 30)

    # Get response from Gemini
    print("\n--- Gemini Response ---")
    gemini_response = get_gemini_response(user_prompt, GEMINI_API_KEY)
    print(gemini_response)
    print("-" * 30)

     # Add results to Google Sheet
    sheet_data = ["Code Explanation", gpt_response, deepseek_response, gemini_response]
    add_to_google_sheet(sheet_data)
    print("Results added to Google Sheet.")


if __name__ == "__main__":
    main()

Welcome to the LLM Code Analyzer!
------------------------------

Getting responses from LLMs...

--- GPT-4o Response ---
```java
public Object streamLoad(HttpServletRequest request,
                         @PathVariable(value = DB_KEY) String db,
                         @PathVariable(value = TABLE_KEY) String table) {
    if (isGroupCommitEnabled(request)) { // Extracted repeated logic into a separate method
        try {
            String[] pair = new String[]{db, table};
            if (isGroupCommitBlock(pair)) {
                return new RestBaseResult("insert table " + pair[1] + " is blocked on schema change");
            }
        } catch (Exception e) {
            // handle
        }
    }
    return null; // Added return null to ensure method always returns an Object
}

@RequestMapping(path = "/api/_http_stream", method = RequestMethod.PUT)
public Object streamLoadWithSql(HttpServletRequest request, HttpServletResponse response) {
    String sql = request.getHeader("sql"